In [1]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

Mounted at /content/drive
/content/drive/MyDrive/gaze_estimation


In [2]:
import os
import time
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression

from hglm.precision_module import precision_module
from loss.h_likelihood_precision import nhll_hetero_precision
from util import make_reproducibility, TensorDataset, convert_to_xyz, mae, make_arbitrary_masking, k_fold_index
from networks import *
from hglm.hglm_hetero_precision import hetero_precision_without_val

In [3]:
#@title Load preprocessed & subsampled data (LOOCV)

ids =                    np.load('../mpii_dataset/within_ids.npy')
images = torch.as_tensor(np.load('../mpii_dataset/within_images.npy'), dtype=torch.float)
hps =    torch.as_tensor(np.load('../mpii_dataset/within_2d_hps.npy'), dtype=torch.float)
gazes =  torch.as_tensor(np.load('../mpii_dataset/within_2d_gazes.npy'), dtype=torch.float)

In [4]:
device = torch.device('cuda:0')
seed = 10
experiment_name = 'mpii_within'

pretrain_iter = 1
m_pretrain_epoch = 50
v_pretrain_epoch = 20

max_iter = 150
mean_epoch = 10
v_step_iter = 100
patience = 10

mean_lr = 5e-3
variance_lr = 1e-3
batch_size = 1000
weight_decay = 0

hidden_features = 500
test_unseen=False
large_test=True

In [5]:
res_list = []

In [6]:
for fold in range(10) :

    train_ids = np.concatenate([ids[:fold], ids[(fold+1):]]).reshape(-1)
    train_images = torch.cat([images[:fold], images[(fold+1):]]).reshape(-1,36,60)
    train_hps = torch.cat([hps[:fold], hps[(fold+1):]]).reshape(-1,2)
    train_gazes = torch.cat([gazes[:fold], gazes[(fold+1):]]).reshape(-1,2)

    test_ids = ids[fold]
    test_images = images[fold]
    test_hps = hps[fold]
    test_gazes = gazes[fold]

    res_list.append(hetero_precision_without_val(
        train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        ResNet_batchnorm.ResNet_batchnorm, hidden_features=hidden_features, K=2, 
        mean_lr=mean_lr, variance_lr=variance_lr, weight_decay=weight_decay, batch_size=batch_size,
        pretrain_iter=pretrain_iter, m_pretrain_epoch=m_pretrain_epoch, v_pretrain_epoch=v_pretrain_epoch, max_iter=max_iter, mean_epoch=mean_epoch, v_step_iter=v_step_iter, patience=patience,
        device=device, experiment_name=f'{experiment_name}_{fold}', SEED=seed + fold,
        normalize=True, deg=True, test_unseen=test_unseen, weighted=True, verbose=False, large_test=large_test))




Output hidden; open in https://colab.research.google.com to view.